In [ ]:
# always check your python version
!python --version

In [ ]:
# AWS SDK for Python, -q flag is for quiet
%pip install -q boto3

In [ ]:
# always check your boto3 version
%pip show boto3

In [ ]:
# Deploy CFN Template

In [ ]:
!pip install - q redis

In [ ]:

client = boto3.client('bedrock-runtime')

In [ ]:
from helper import load_text_file, store_conversation, get_conversation

def converse(session_id, input):
  history = get_conversation(session_id)  

  system_prompt = load_text_file("prompts/jp-teacher.txt")
  model_id = 'anthropic.claude-v2'
  prompt = f"{system_prompt}\n{history}\nUser: ${input}\nAssistant:"
  resp = client.invoke_model(
      modelId=model_id,
      contentType='application/json',
      accept='application/json',
      body=json.dumps({
        "prompt": prompt,
      })
  )
  body = json.loads(resp.get("body").read())
  llm_response = body['completion']

  updated_history = f"{history}\n{llm_response}"
  store_conversation(session_id,updated_history)

  return llm_response

In [ ]:
print(converse("Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

In [ ]:
print(converse("How do I conjugate taberu?"))

In [ ]:
class JpAgent:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [] # load with initial messages
  def chat(messages, input):
    system_prompt = load_text_file("prompts/jp-teacher.txt")
    model_id = 'anthropic.claude-v2'
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse(
        modelId=model_id,
        messages=self.messages,
        system=system_prompt
    )

In [ ]:
agent = JpAgent()
print(agent.chat("Let's go eat food at the resturant Kyoto Katsugyu in Toronto."))

In [ ]:
print(agent.chat("How do I conjugate taberu?"))